In [2]:
import sqlite3

In [3]:
class DatabaseGestion:
    def __init__(self, path='../data/olist.db'):
        self.db_path = path
        self.conn = sqlite3.connect(self.db_path)

    def do_request(self, sql_request):
        try:
            cursor = self.conn.cursor()

            # Faire la requete
            result = cursor.execute(sql_request)

            # Recuperer le resultat
            return list(result)

        except Exception as error:
            print(f"An error occurred: {error}")
            return None


In [4]:
DG = DatabaseGestion()

In [5]:
# Afficher le noms des tables dans le fichier db
DG.do_request("SELECT name FROM sqlite_master WHERE type = 'table'")

[('customers',),
 ('geoloc',),
 ('order_items',),
 ('order_pymts',),
 ('order_reviews',),
 ('orders',),
 ('products',),
 ('sellers',),
 ('translation',)]

In [6]:
# Afficher tous les champs disponible pour une table donnée
DG.do_request("PRAGMA table_info(customers);")

[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'customer_id', 'TEXT', 0, None, 0),
 (2, 'customer_unique_id', 'TEXT', 0, None, 0),
 (3, 'customer_zip_code_prefix', 'BIGINT', 0, None, 0),
 (4, 'customer_city', 'TEXT', 0, None, 0),
 (5, 'customer_state', 'TEXT', 0, None, 0)]

# Requete 1
En excluant les commandes annulées, quelles sont les commandes
récentes de moins de 3 mois que les clients ont reçues avec au moins 3
jours de retard ?

In [7]:
# Afficher les champs de la table orders
DG.do_request("PRAGMA table_info(orders);")


[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'order_id', 'TEXT', 0, None, 0),
 (2, 'customer_id', 'TEXT', 0, None, 0),
 (3, 'order_status', 'TEXT', 0, None, 0),
 (4, 'order_purchase_timestamp', 'TEXT', 0, None, 0),
 (5, 'order_approved_at', 'TEXT', 0, None, 0),
 (6, 'order_delivered_carrier_date', 'TEXT', 0, None, 0),
 (7, 'order_delivered_customer_date', 'TEXT', 0, None, 0),
 (8, 'order_estimated_delivery_date', 'TEXT', 0, None, 0)]

In [8]:
# Afficher toutes les valeurs unique du champ orders
DG.do_request("SELECT DISTINCT order_status FROM orders ORDER BY order_status ")

[('approved',),
 ('canceled',),
 ('created',),
 ('delivered',),
 ('invoiced',),
 ('processing',),
 ('shipped',),
 ('unavailable',)]

In [9]:
# Afficher la commande la plus recente pour avoir une date de reference
DG.do_request("SELECT MAX(order_purchase_timestamp) AS date_la_plus_recente FROM orders;")

[('2018-10-17 17:30:18',)]

In [10]:
# Selection de toutes les commandes de moins de 4 mois a compté du  2018-10-17 17:30:18 et qui ne sont pas cancel
DG.do_request("""
            SELECT order_id,order_status,order_purchase_timestamp 
            FROM orders 
            WHERE order_purchase_timestamp >= date('2018-10-17 17:30:18', '-3 months') 
            AND order_purchase_timestamp <= date('2018-10-17 17:30:18') 
            AND order_status != 'canceled';
""")

[('53cdb2fc8bc7dce0b6741e2150273451', 'delivered', '2018-07-24 20:41:37'),
 ('47770eb9100c2d0c44946d9cf07ec65d', 'delivered', '2018-08-08 08:38:49'),
 ('5ff96c15d0b717ac6ad1f3d77225a350', 'delivered', '2018-07-25 17:44:10'),
 ('f3e7c359154d965827355f39d6b1fdac', 'delivered', '2018-08-09 11:44:40'),
 ('b276e4f8c0fb86bd82fce576f21713e0', 'delivered', '2018-07-29 23:34:51'),
 ('d22e9fa5731b9e30e8b27afcdc2f8563', 'delivered', '2018-08-04 23:25:30'),
 ('5820a1100976432c7968a52da59e9364', 'delivered', '2018-07-29 11:24:17'),
 ('9faeb9b2746b9d7526aef5acb08e2aa0', 'delivered', '2018-07-26 14:39:59'),
 ('f346ad4ee8f630e5e4ddaf862a34e6dd', 'delivered', '2018-08-05 13:09:48'),
 ('bd4bd0194d6d29f83b8557d4b89b572a', 'delivered', '2018-07-28 16:52:55'),
 ('6d25592267349b322799e2beb687871e', 'delivered', '2018-08-26 22:04:34'),
 ('77e9941864fc840be8e4b1ba5347c0f7', 'delivered', '2018-08-03 08:59:39'),
 ('e3447938231fb6ba2fee0231b51eca59', 'delivered', '2018-08-26 11:04:26'),
 ('a5474c0071dd5d1074e12d

In [11]:
# Selection de toutes les commandes de moins de 3 mois a compté du  2018-10-17 17:30:18 et qui ne sont pas cancel recu au minimum 3j plus tard
DG.do_request("""
            SELECT order_status,order_estimated_delivery_date,order_delivered_customer_date
            FROM orders 
            WHERE order_purchase_timestamp >= date('2018-10-17 17:30:18', '-3 months') AND order_purchase_timestamp <= date('2018-10-17 17:30:18')  --- On verifie que les commandes sont bien de moins de 3mois
            AND order_status != 'canceled'
            AND order_estimated_delivery_date <= date(order_delivered_customer_date, '-3 days') --- Calcul du delta de temps entre la livraison prevu et la livraison reel
;""")

[('delivered', '2018-08-22 00:00:00', '2018-08-29 01:41:41'),
 ('delivered', '2018-08-23 00:00:00', '2018-09-01 18:14:42'),
 ('delivered', '2018-08-03 00:00:00', '2018-08-14 04:04:40'),
 ('delivered', '2018-08-09 00:00:00', '2018-08-13 20:11:47'),
 ('delivered', '2018-08-17 00:00:00', '2018-08-21 00:11:52'),
 ('delivered', '2018-08-10 00:00:00', '2018-08-14 17:21:32'),
 ('delivered', '2018-08-10 00:00:00', '2018-08-14 17:51:20'),
 ('delivered', '2018-08-14 00:00:00', '2018-08-22 14:15:51'),
 ('delivered', '2018-08-13 00:00:00', '2018-08-20 15:55:42'),
 ('delivered', '2018-08-13 00:00:00', '2018-08-24 17:03:18'),
 ('delivered', '2018-08-17 00:00:00', '2018-08-21 18:19:01'),
 ('delivered', '2018-08-10 00:00:00', '2018-08-15 17:58:41'),
 ('delivered', '2018-08-08 00:00:00', '2018-08-13 16:54:31'),
 ('delivered', '2018-08-14 00:00:00', '2018-08-21 11:28:49'),
 ('delivered', '2018-08-09 00:00:00', '2018-08-13 16:12:25'),
 ('delivered', '2018-08-23 00:00:00', '2018-08-28 01:07:29'),
 ('deliv

# Requete 2
Qui sont les vendeurs ayant généré un chiffre d'affaires de plus de 100
000 Real sur des commandes livrées via Olist ?

In [12]:
# Afficher tous les champs disponible pour une table donnée
DG.do_request("PRAGMA table_info(order_items);")

[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'order_id', 'TEXT', 0, None, 0),
 (2, 'order_item_id', 'BIGINT', 0, None, 0),
 (3, 'product_id', 'TEXT', 0, None, 0),
 (4, 'seller_id', 'TEXT', 0, None, 0),
 (5, 'shipping_limit_date', 'TEXT', 0, None, 0),
 (6, 'price', 'FLOAT', 0, None, 0),
 (7, 'freight_value', 'FLOAT', 0, None, 0)]

In [13]:
DG.do_request("""
SELECT 
    seller_id, 
    SUM(price) AS total_revenue  --- Création d'une variable total_revenu qui permet de connaitre le montant total des ventes d'un vendeur par la suite
FROM 
    order_items
JOIN 
    orders ON order_items.order_id = orders.order_id  --- Combiner les données de orders et order_item grace a leur id commun
WHERE 
    orders.order_status = 'delivered' 
GROUP BY 
    seller_id
HAVING 
    total_revenue > 100000;
""")

[('1025f0e2d44d7041d6cf58b6550e0bfa', 138208.56),
 ('1f50f920176fa81dab994f9023523100', 106655.70999999999),
 ('46dc3b2cc0980fb8ec44634e21d2718e', 122811.38),
 ('4869f7a5dfa277a7dca6462dcf3b52b2', 226987.93),
 ('4a3ca9315b744ce9f8e9374361493884', 196882.12),
 ('53243585a1d6dc2643021fd1853d8905', 217940.44),
 ('5dceca129747e92ff8ef7a997dc4f8ca', 111126.73),
 ('620c87c171fb2a6dd6e8bb4dec959fc6', 112461.5),
 ('6560211a19b47992c3666cc44a7e94c0', 120702.83),
 ('7a67c85e85bb2ce8582c35f2203ad736', 139658.69),
 ('7c67e1448b00f6e969d365cea6b010ab', 186570.05),
 ('7d13fca15225358621be4086e1eb0964', 112436.18000000001),
 ('7e93a43ef30c4f03f38b393420bc753a', 165981.49),
 ('955fee9216a65b617aa5c0531780ce60', 131836.71),
 ('cc419e0650a3c5ba77189a1882b7556a', 101090.95999999999),
 ('da8622b14eb17ae2831f4ac5b9dab84a', 159816.87),
 ('fa1c13f2614d7b5c4749cbc52fecda94', 190917.14)]

# Requete 3 
Qui sont les nouveaux vendeurs (moins de 3 mois d'ancienneté) qui
sont déjà très engagés avec la plateforme (ayant déjà vendu plus de 30
produits) ?

In [14]:
# Afficher tous les champs disponible pour une table donnée
DG.do_request("PRAGMA table_info(sellers);")

[(0, 'index', 'BIGINT', 0, None, 0),
 (1, 'seller_id', 'TEXT', 0, None, 0),
 (2, 'seller_zip_code_prefix', 'BIGINT', 0, None, 0),
 (3, 'seller_city', 'TEXT', 0, None, 0),
 (4, 'seller_state', 'TEXT', 0, None, 0)]

In [15]:
DG.do_request("""
SELECT


""")

An error occurred: incomplete input


In [20]:
DG.do_request("""
WITH FirstSaleDate AS (
    SELECT 
        oi.seller_id,
        MIN(o.order_purchase_timestamp) AS first_sale_date
    FROM 
        order_items oi
    INNER JOIN 
        orders o ON oi.order_id = o.order_id
    GROUP BY 
        oi.seller_id
),
RecentSellers AS (
    SELECT 
        seller_id
    FROM 
        FirstSaleDate
    WHERE 
        DATE('now') - DATE(first_sale_date) <= 90
),
SellerSales AS (
    SELECT 
        seller_id,
        COUNT(order_item_id) AS total_products_sold
    FROM 
        order_items
    GROUP BY 
        seller_id
)
SELECT 
    rs.seller_id
FROM 
    RecentSellers rs
INNER JOIN 
    SellerSales ss ON rs.seller_id = ss.seller_id
WHERE 
    ss.total_products_sold > 30;



""")

[('001cca7ae9ae17fb1caed9dfb1094831',),
 ('002100f778ceb8431b7a1020ff7ab48f',),
 ('004c9cd9d87a3c30c522c48c4fc07416',),
 ('00ee68308b45bc5e2660cd833c3f81cc',),
 ('00fc707aaaad2d31347cf883cd2dfe10',),
 ('0176f73cc1195f367f7b32db1e5b3aa8',),
 ('01cf7e3d21494c41fb86034f2e714fa1',),
 ('01fd077212124329bac32490e8ef80d9',),
 ('01fdefa7697d26ad920e9e0346d4bd1b',),
 ('0241d4d5d36f10f80c644447315af0bd',),
 ('02f623a8eb246f3c5f7c2f96462654e6',),
 ('04308b1ee57b6625f47df1d56f00eedf',),
 ('0432ead42b6c8a0bdf68154add917fdf',),
 ('048c2757535328e0d7dac690ad3c0aae',),
 ('0509040ea3fe50071181bbc359eb7738',),
 ('056b4ada5bbc2c50cc7842547dda6b51',),
 ('058fd0aa2bfdb2274e05e1ae971dabb6',),
 ('05d2173d43ea568aa0540eba70d2ca76',),
 ('05f51e13da97139648b8125c31e5f51b',),
 ('05ff92fedb5be47920fea08e501238b9',),
 ('062ce95fa2ad4dfaedfc79260130565f',),
 ('06579cb253ecd5a3a12a9e6eb6bf8f47',),
 ('066a6914e1ebf3ea95a216c73a986b91',),
 ('0691148aee60ca47977c187804f935ae',),
 ('06a2c3af7b3aee5d69171b0e14f0ee87',),


Fonctionnement de la requete

- FirstSaleDate (Sous-requête 1) :
Cette étape identifie la première vente pour chaque vendeur en utilisant la date la plus ancienne de l'achat des commandes (order_purchase_timestamp). Elle sélectionne seller_id et la first_sale_date en utilisant la jointure entre order_items et orders, et regroupe par seller_id.

- RecentSellers (Sous-requête 2) :
Cette étape sélectionne les vendeurs dont la première vente a eu lieu dans les 90 derniers jours. Elle utilise la table FirstSaleDate et compare la différence entre la date actuelle et la first_sale_date.

- SellerSales (Sous-requête 3) :
Cette étape calcule le nombre total de produits vendus par chaque vendeur. Elle compte les occurrences de order_item_id dans la table order_items et regroupe par seller_id.

Sélection finale :
Cette étape combine les résultats des sous-requêtes RecentSellers et SellerSales en une jointure interne (INNER JOIN). Elle sélectionne les seller_id des vendeurs récents (RecentSellers) et dont le nombre total de produits vendus est supérieur à 30, filtrant les résultats avec la condition ss.total_products_sold > 30.

La requête retourne donc les identifiants des vendeurs qui ont effectué leur première vente dans les 90 derniers jours et ont vendu plus de 30 produits.

# Requete 4
Quels sont les 5 codes postaux, enregistrant plus de 30
reviews, avec le pire review score moyen sur les 12 derniers mois ?